# Create MBTiles from vector type data

This notebook transforms vector type data into `MBTiles` using tippecanoe.

## Table of Contents
### [Python libraries](#libraries)
### [Utils](#utils)
- **[create_mbtiles](#create_mbtiles)**

### [Read data](#carto)
- **[Indigenous Territories](#indigenous_territories)**
- **[Land Rights](#land_rights)**
- **[Major Mineral Deposits](#mineral_deposits)**

### [Create `MBTiles`](#create_mbtiles_2)
### [Show `MBTiles` in our localhost](#show_mbtiles)

<a id='libraries'></a>
### Python libraries

In [42]:
import json
import requests
import numpy as np
import geopandas as gpd
import subprocess

<a id='utils'></a>
### Utils

<a id='create_mbtiles'></a>
**create_mbtiles**

In [2]:
def create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel"):
    """
    Use tippecanoe to create a MBTILE at dest_path from source_path.
    layer_name is used for the name of the layer in the MBTILE.
    Regex file path (/*.geojson) is supported for source_path.
    """
    cmd = f"tippecanoe -o {dest_path} -l {layer_name} {opts} {source_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

<a id='carto'></a>
### Read data 

<a id='indigenous_territories'></a>
**[Indigenous Territories](https://native-land.ca/)**

In [ ]:
url = 'https://native-land.ca/coordinates/indigenousTerritories.json'

data = requests.get(url).json()

**Save data as `JSON`**

In [ ]:
with open('../data/mbtiles/Indigenous_Territories.json', 'w') as fp:
    json.dump(data, fp)

<a id='land_rights'></a>
**[Land Rights](http://data.globalforestwatch.org/datasets/e22b22b3525c46f3b35548daf6289903_1/data)**

In [16]:
data = gpd.read_file('../data/mbtiles/gfw_land_rights/gfw_land_rights.shp')
data['geometry'] = data['geometry'].apply(lambda x: x.simplify(0.0075))
data.head()

,country,name,legal_term,legal_reco,area_ha,source,last_updat,gfwid,st_area_sh,st_length_,geometry
0,CAN,ENGLISH RIVER HAULTAIN LAKE I.R. 192K,Indian Reserve,Y,201.288027174093,"Government of Canada, Natural Resources Canada...",2014,c531a195-09d5-449c-84fb-129d7bb77901,6.703042e+06,11842.772655,"POLYGON ((-11833624.371 7725300.176, -11832861..."
1,CAN,ELAK DASE I.R. 192A,Indian Reserve,Y,1397.67492142709,"Government of Canada, Natural Resources Canada...",2014,47f27ab3-f33b-407a-91dc-39667821c216,4.429801e+07,40679.063996,"POLYGON ((-11887246.473 7528355.931, -11887244..."
2,CAN,MIN-A-HE-QUO-SIS INDIAN RESERVE NO. 116A,Indian Reserve,Y,260.538624047784,"Government of Canada, Natural Resources Canada...",2014,0c445e83-c893-4fb3-9efb-efd0519a3df6,7.207503e+06,10749.287035,"POLYGON ((-12139665.105 6999456.605, -12138323..."
3,CAN,TSU TUE 196G,Indian Reserve,Y,42.7328901923613,"Government of Canada, Natural Resources Canada...",2014,349f8363-39ed-480b-bfd7-4228f5f3f4f1,1.698421e+06,5973.105602,"POLYGON ((-12307276.499 8389447.707, -12307276..."
4,CAN,WHITE FISH LAKE 128,Indian Reserve,Y,6156.88101888983,"Government of Canada, Natural Resources Canada...",2014,502a3edf-d25c-4121-863f-2a17dd1ab4d4,1.805401e+08,100897.806092,"POLYGON ((-12445854.141 7221022.378, -12445888..."


In [18]:
data = data.to_crs("EPSG:4326")
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

**Save data as `JSON`**

In [22]:
data.to_file('../data/mbtiles/Land_Rights.json', driver="GeoJSON")

<a id='mineral_deposits'></a>
**[Major Mineral Deposits](https://mrdata.usgs.gov/major-deposits/)**

In [8]:
data = gpd.read_file('../data/mbtiles/major_mineral_deposits/ofr20051294.shp')
data.head()

,REC_ID,DEP_NAME,COUNTRY,STATE,COMMODITY,DEP_TYPE,DETAIL,MODEL,DEP_MODEL,CITATION,CATEGORY,URL,geometry
0,325,Wando,Korean Peninsula,None,Pyrophyllite,Hydrothermal,None,None,None,"O'Driscoll, 1993",None,https://mrdata.usgs.gov/major-deposits/show-of...,POINT (126.68330 34.35000)
1,52,Georgina Basin,Australia,None,Phosphorous,Sedimentary,None,None,None,"Industrial Minerals, 1998",None,https://mrdata.usgs.gov/major-deposits/show-of...,POINT (139.96670 -21.88330)
2,1880,Wheal Jane,United Kingdom,None,Tin,Hydrothermal,None,None,None,None,None,https://mrdata.usgs.gov/major-deposits/show-of...,POINT (-5.20000 50.23333)
3,1979,Kaldygajtinskoe,Kazakhstan,None,Halite,Sedimentary,None,None,None,None,None,https://mrdata.usgs.gov/major-deposits/show-of...,POINT (53.60583 49.80639)
4,2548,Imiter,Morocco,None,Silver,Hydrothermal,None,None,None,None,None,https://mrdata.usgs.gov/major-deposits/show-of...,POINT (-5.83360 31.35000)


**Save data as `JSON`**

In [9]:
data.to_file('../data/mbtiles/Major_Mineral_Deposits.json', driver="GeoJSON")

**Trase soy municipalities 2018**

In [72]:
data = gpd.read_file('../data/mbtiles/trase_soy_municipalities_2018.geojson', driver="GeoJSON")
#data['geometry'] = data['geometry'].apply(lambda x: x.simplify(0.0075))
data.head()

,NM_MUNICIP,CD_GEOCMU,BRAZIL_SOY_2.5.1_TRASE_YEAR,BRAZIL_SOY_2.5.1_TRASE_COUNTRY OF PRODUCTION,BRAZIL_SOY_2.5.1_TRASE_BIOME,BRAZIL_SOY_2.5.1_TRASE_STATE,BRAZIL_SOY_2.5.1_TRASE_LOGISTICS HUB,BRAZIL_SOY_2.5.1_TRASE_PORT,BRAZIL_SOY_2.5.1_TRASE_EXPORTER,BRAZIL_SOY_2.5.1_TRASE_EXPORTER GROUP,BRAZIL_SOY_2.5.1_TRASE_IMPORTER,BRAZIL_SOY_2.5.1_TRASE_IMPORTER GROUP,BRAZIL_SOY_2.5.1_TRASE_COUNTRY,BRAZIL_SOY_2.5.1_TRASE_ECONOMIC BLOC,BRAZIL_SOY_2.5.1_TRASE_TYPE,BRAZIL_SOY_2.5.1_TRASE_SOY_EQUIVALENT_TONNES,BRAZIL_SOY_2.5.1_TRASE_FOB_USD,BRAZIL_SOY_2.5.1_TRASE_LAND_USE_HA,BRAZIL_SOY_2.5.1_TRASE_TERRITORIAL_DEFORESTATION_RISK_HA,geometry
0,ABADIA DE GOIÁS,5200050,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-49.44440 -16.71747, -49.44435..."
1,ABADIA DOS DOURADOS,3100104,2018,BRAZIL,CERRADO,MINAS GERAIS,UBERLANDIA,VITORIA,GAVILON,GAVILON,UNKNOWN,UNKNOWN,CHINA (MAINLAND),CHINA (MAINLAND),Soy bean equivalents,34.65094138,13610.5551101655,8.75023772222222,0.288328008594347,"MULTIPOLYGON (((-47.43840 -18.16570, -47.43588..."
2,ABADIÂNIA,5200100,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-48.84178 -16.00488, -48.79639..."
3,ABAETÉ,3100203,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-45.16777 -18.89073, -45.16719..."
4,ABAETETUBA,1500107,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-48.83139 -1.56352, -48.82910 ..."


In [73]:
data.fillna(value=np.nan, inplace=True)
data['BRAZIL_SOY_2.5.1_TRASE_YEAR'] = data['BRAZIL_SOY_2.5.1_TRASE_YEAR'].fillna(value=-9999)
#data['BRAZIL_SOY_2.5.1_TRASE_TERRITORIAL_DEFORESTATION_RISK_HA'] = data['BRAZIL_SOY_2.5.1_TRASE_TERRITORIAL_DEFORESTATION_RISK_HA'].replace('', np.nan)

In [74]:
data.head()

,NM_MUNICIP,CD_GEOCMU,BRAZIL_SOY_2.5.1_TRASE_YEAR,BRAZIL_SOY_2.5.1_TRASE_COUNTRY OF PRODUCTION,BRAZIL_SOY_2.5.1_TRASE_BIOME,BRAZIL_SOY_2.5.1_TRASE_STATE,BRAZIL_SOY_2.5.1_TRASE_LOGISTICS HUB,BRAZIL_SOY_2.5.1_TRASE_PORT,BRAZIL_SOY_2.5.1_TRASE_EXPORTER,BRAZIL_SOY_2.5.1_TRASE_EXPORTER GROUP,BRAZIL_SOY_2.5.1_TRASE_IMPORTER,BRAZIL_SOY_2.5.1_TRASE_IMPORTER GROUP,BRAZIL_SOY_2.5.1_TRASE_COUNTRY,BRAZIL_SOY_2.5.1_TRASE_ECONOMIC BLOC,BRAZIL_SOY_2.5.1_TRASE_TYPE,BRAZIL_SOY_2.5.1_TRASE_SOY_EQUIVALENT_TONNES,BRAZIL_SOY_2.5.1_TRASE_FOB_USD,BRAZIL_SOY_2.5.1_TRASE_LAND_USE_HA,BRAZIL_SOY_2.5.1_TRASE_TERRITORIAL_DEFORESTATION_RISK_HA,geometry
0,ABADIA DE GOIÁS,5200050,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-49.44440 -16.71747, -49.44435..."
1,ABADIA DOS DOURADOS,3100104,2018,BRAZIL,CERRADO,MINAS GERAIS,UBERLANDIA,VITORIA,GAVILON,GAVILON,UNKNOWN,UNKNOWN,CHINA (MAINLAND),CHINA (MAINLAND),Soy bean equivalents,34.65094138,13610.5551101655,8.75023772222222,0.288328008594347,"MULTIPOLYGON (((-47.43840 -18.16570, -47.43588..."
2,ABADIÂNIA,5200100,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-48.84178 -16.00488, -48.79639..."
3,ABAETÉ,3100203,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-45.16777 -18.89073, -45.16719..."
4,ABAETETUBA,1500107,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-48.83139 -1.56352, -48.82910 ..."


In [75]:
data = data.astype({'CD_GEOCMU': int, 'BRAZIL_SOY_2.5.1_TRASE_YEAR': int,
                   #'BRAZIL_SOY_2.5.1_TRASE_SOY_EQUIVALENT_TONNES': float,
                   #'BRAZIL_SOY_2.5.1_TRASE_FOB_USD': float,
                   'BRAZIL_SOY_2.5.1_TRASE_LAND_USE_HA': float,
                   #'BRAZIL_SOY_2.5.1_TRASE_TERRITORIAL_DEFORESTATION_RISK_HA': float
                   })

In [76]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 20 columns):
 #   Column                                                    Non-Null Count  Dtype   
---  ------                                                    --------------  -----   
 0   NM_MUNICIP                                                5572 non-null   object  
 1   CD_GEOCMU                                                 5572 non-null   int64   
 2   BRAZIL_SOY_2.5.1_TRASE_YEAR                               5572 non-null   int64   
 3   BRAZIL_SOY_2.5.1_TRASE_COUNTRY OF PRODUCTION              1411 non-null   object  
 4   BRAZIL_SOY_2.5.1_TRASE_BIOME                              1411 non-null   object  
 5   BRAZIL_SOY_2.5.1_TRASE_STATE                              1411 non-null   object  
 6   BRAZIL_SOY_2.5.1_TRASE_LOGISTICS HUB                      1411 non-null   object  
 7   BRAZIL_SOY_2.5.1_TRASE_PORT                               1411 non-null   object  
 8   

**Save data as `JSON`**

In [77]:
data.to_file('../data/mbtiles/trase_soy_municipalities_2018.json', driver="GeoJSON")

<a id='create_mbtiles_2'></a>
### Create `MBTiles`

In [78]:
layers = {"Indigenous Territories": 'Indigenous_Territories',
         "Land Rights": 'Land_Rights',
         "Major Mineral Deposits": 'Major_Mineral_Deposits'}

layers = {"Trase Soy Municipalities": 'trase_soy_municipalities_2018'}

for layer_name, file in layers.items():

    source_path = f"../data/mbtiles/{file}.json"
    dest_path = f"../data/mbtiles/{file}.mbtiles"
    create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")

Processing: tippecanoe -o ../data/mbtiles/trase_soy_municipalities_2018.mbtiles -l Trase Soy Municipalities -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel ../data/mbtiles/trase_soy_municipalities_2018.json
Task created
Finished processing


<a id='show_mbtiles'></a>
### Show `MBTiles` in our localhost

In [79]:
!mbview --port 9000 ../data/mbtiles/trase_soy_municipalities_2018.mbtiles

Listening on http://localhost:9000
^C
